In [2]:
#计算图是现代深度学习框架的核心
#手动求取反向传播废时费力，而且容易出错
#autograd.Variable封装了data,grad,grad_fn三个数据
#data是tensor，grad的形状与data一致，grad_fn指向一个Func记录操作历史

#可选参数
#requires_grad(bool):是否对该ariable求导数
#volatile(bool):设置为True，构建在该variable上的图都不会求道，为推理阶段设计


In [10]:
import torch as t
from torch.autograd import Variable as var

#从tensor中创建var，指定需要求导
a = var(t.ones(3,4),requires_grad = True)
b = var(t.zeros(3,4))
print(a)
print(b)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], requires_grad=True)
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


In [15]:
c = a+b
print(c)
c = a.add(b)
print(c)
#上面两种方法一样结果

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], grad_fn=<AddBackward0>)
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], grad_fn=<AddBackward0>)


In [26]:
d = c.sum()
d.backward()#反向传播

In [29]:
#注意区别，第一种因为先取了data部分再sum，所以数据类型是tensor
c.data.sum()
#第二种因为直接sum,所以数据类型仍然为var
c.sum()

tensor(12., grad_fn=<SumBackward0>)